In [2]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
def last_price(market_id):

    url = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
    response = requests.get(url)
    ticker_info = response.json()
    return float(ticker_info['ticker']['last_price'][0])

def precio_ponderado_order(df, x):
    '''funcion calcula el precio ponderado si quiero ejecutar x cantidad
    df: dataframe con ordenes
    x: cantidad de cripto'''
    
    _df = df[df['Acum_vol']<= x].append(df[df['Acum_vol']>= x].head(1))
    _df.iloc[-1, _df.columns.get_loc('Acum_vol')] = x
    _df.iloc[-1, _df.columns.get_loc('volume')] = x - _df.iloc[-2]['Acum_vol']

    return ((_df['price'] * _df['volume']).sum()) / (_df['volume'].sum())

class Market():

    def __init__(self, market_id):
        self.id = market_id
        self.price = last_price(self.id)
        self.orders = False
        
    def __str__(self):
        return ('mercado %s Last Price: %s' %(self.id, self.price) )

    def last_price(self):
        self.price = last_price(self.id)
        return self.price
    
    def order_book(self, tipo=None):
        '''
        funcion obtiene libro de ordenes
        tipo: [asks, bids]. None entregara ambos libros
        '''
        market_id = self.id
        self.orders = pd.DataFrame()
        url = f'https://www.buda.com/api/v2/markets/{market_id}/order_book'
        response = requests.get(url)
        order_book = response.json()['order_book']
        columnas = ['price', 'volume']
        df_a = pd.DataFrame(order_book['asks'], columns=columnas).astype(float)
        df_a['tip'] = 'asks'
        df_a.set_index('tip', inplace=True)
        df_b = pd.DataFrame(order_book['bids'], columns=columnas).astype(float)
        df_b['tip'] = 'bids'
        df_b.set_index('tip', inplace=True)
        
        self.orders = pd.concat([df_a, df_b])

        if tipo == 'asks':
            return df_a
        elif tipo == 'bids':
            return df_b
        else:
            return self.orders

    def market_liquidity(self):
        
        self.order_book()
        
        # Suma Volumnes tottales agrupa en bid y en ask
        liqdty_idx = pd.pivot_table(self.orders, index=self.orders.index, values='volume', aggfunc=np.sum)
        # liqdty_idx['Precio_Ponderado'] = self.orders.groupby(self.orders.index).apply(lambda x: np.average(x['price'], weights=x['volume']))
        # Separo libros por tipo y ordeno por precio
        df_ask = self.orders.loc['asks'].sort_values('price')
        df_bid = self.orders.loc['bids'].sort_values('price', ascending=False)

        # creo columna de suma acumulada
        df_ask['Acum_vol'] = df_ask['volume'].cumsum()
        df_bid['Acum_vol'] = df_bid['volume'].cumsum()
        
        for x in np.linspace(0.1,1,10):
            # Creo rangos 10%, 20%... 100% del volumnes total
            x_ask = liqdty_idx['volume']['asks'] * x
            x_bid = liqdty_idx['volume']['bids'] * x
            # cAculo precio ponderado y guardo en liqdty_idx
            liqdty_idx['wght_price'+str(int(x*100)) +'%'] = [precio_ponderado_order(df_ask, x_ask), precio_ponderado_order(df_bid, x_bid)]

        return liqdty_idx

    def market_cap(self):

        self.order_book()

        # Volumne total * last price
        cap_a = self.orders.loc['asks']['volume'].sum() * self.last_price()
        cap_b = self.orders.loc['bids']['volume'].sum() * self.last_price()
        return ({'asks':cap_a, 'bids': cap_b})


    def cantidad_mercado_impacto(self, q, tipo = 'buy'):
        '''funcion 
        devuelve la cantidad q hay q vender/comprar para que el precio cambie en q%
        '''
        self.order_book()
        
        if tipo == 'buy':
            df = self.orders.loc['asks']
            df['Cumulative'] = df['volume'].cumsum()
            price_limit = self.price * (1 + q)
            df = df[df['price']>=price_limit]
        else:
            df = self.orders.loc['bids']
            df['Cumulative'] = df['volume'].cumsum()
            price_limit = self.price * (1 - q)
            df = df[df['price']<=price_limit]

        return df['Cumulative'].values[0]

    def market_depth(self, tipo='asks'):
        df_a = self.orders.loc[tipo]
        df_a['Cumulative'] = df_a['volume'].cumsum()
        df_a['change'] = df_a['price'] / self.price -1

        # df_b = self.orders.loc['bids']
        # df_b['Cumulative'] = df_b['volume'].cumsum()
        # df_b['change'] = df_b['price'] / self.price -1

        data = {}
        for i in np.linspace(0.05,1,20):
            if tipo == 'asks':
                mount = df_a[df_a['change']>=i]['Cumulative'].values[0]
            elif tipo =='bids':
                mount = df_a[df_a['change']<=-i]['Cumulative'].min()
            # data.append({tipo:{i:mount}})
            data[i]= [mount]
            # data.append({'bid':{i:mount_b}})

        return data

    def precio_fin_order(self, p, tipo='asks'):
        '''
        devuelve el precio final que quedara el mercado despues de ejecutar p unidades
        '''
        df = self.orders.loc[tipo]
        df['Cumulative'] = df['volume'].cumsum()
        
        df = df[df['Cumulative']>=p]
        price = df['price'].values[0]
        return {'price:': price, 'Change:': price /self.price -1}

### Creamos el mercado que vamos a analizar

In [4]:
btc = Market('btc-clp')
btc.order_book() # Descargamos libro de ordenes
print('precio: '+ str(btc.price))

precio: 51589227.0


##### Suma Total de ordenes (asks y bids)

In [5]:
btc.market_liquidity()[['volume']]

,volume
tip,
asks,37.924276
bids,173.265874


##### Cantidad de BTC que hay q ejecutar para cambiar el precio en distintos proporciones

In [6]:
# podemos obtener la cantidad necesaria de bitcoin q hay que comprar para mover el precio en un 5%
btc.cantidad_mercado_impacto(0.05, 'buy')

13.934063989999999

In [7]:
# Tabla con distintos rangos de cambios porcentuales
pd.concat([pd.DataFrame(btc.market_depth('asks'), index=['asks']),pd.DataFrame(btc.market_depth('bids'), index=['bids'])])


,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1.00
asks,13.934064,17.951404,19.943406,22.340544,22.843819,23.814000,23.871092,25.396133,25.702814,26.072219,26.36751,27.316031,27.323217,27.342972,27.738330,27.792835,28.064347,28.570559,30.044242,30.044242
bids,6.897998,14.346565,20.291564,22.698306,26.078601,28.712883,35.652167,38.462039,41.825089,43.624634,44.87391,46.402785,48.126624,60.702042,62.250174,63.731578,68.887574,70.228074,70.759252,NaN


#### precio final si vendemos x BTC

In [8]:
btc.precio_fin_order(5, 'bids')

{'price:': 49532000.0, 'Change:': -0.03987706580678174}

In [9]:
df = btc.orders.loc['bids']
df['Cumulative'] = df['volume'].cumsum()
precios_list = [[0, btc.price]]
for i in np.arange(0.5, 20, 0.5):
    _df = df[df['Cumulative']<= i]
    precios_list.append([i, _df['price'].min()]) #min con bids max con asks

df = pd.DataFrame(precios_list, columns=['Cantidad_BTC','Precio'])
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['Cantidad_BTC'], y=df['Precio'], name="Precio"),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="Impacto en el mercado"
)

# Set x-axis title
fig.update_xaxes(title_text="Monto BTC")

# Set y-axes titles
fig.update_yaxes(title_text="Precio", secondary_y=False)
fig.update_yaxes(title_text="cambio%", secondary_y=True)

fig.show()